In [25]:
from enum import Enum
import pickle
import re
from functools import lru_cache, reduce, wraps
import itertools
from collections import Counter
import numpy as np

sample = """O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#...."""

sample_tilted_west = """O....#....
OOO.#....#
.....##...
OO.#OO....
OO......#.
O.#O...#.#
O....#OO..
O.........
#....###..
#OO..#...."""

sample_tilted_east = """....O#....
.OOO#....#
.....##...
.OO#....OO
......OO#.
.O#...O#.#
....O#..OO
.........O
#....###..
#..OO#...."""

sample_tilted_north = """OOOO.#.O..
OO..#....#
OO..O##..O
O..#.OO...
........#.
..#....#.#
..O..#.O.O
..O.......
#....###..
#....#...."""

def get_input(n):
    with open('input_'+n+'.txt', 'r') as infile:
        return infile.read().strip()

def parse_input(puzzle_input):
    return np.array([[x for x in l] for l in puzzle_input.split('\n')])

def np_cache(function):
    @lru_cache()
    def cached_wrapper(hashable_array,number):
        array = pickle.loads(hashable_array)
        return function(array,number)

    @wraps(function)
    def wrapper(array,number):
        return cached_wrapper(pickle.dumps(array),number)

    # copy lru_cache attributes over too
    wrapper.cache_info = cached_wrapper.cache_info
    wrapper.cache_clear = cached_wrapper.cache_clear

    return wrapper

def load(dish):
    rocks = dish=='O'
    per_line = rocks.sum(axis=1)
    factors = (np.arange(rocks.shape[1])+1)[::-1]
    return per_line.dot(factors).sum()

assert load(parse_input(sample_tilted_north)) == 136

def tilt_line(line):
    rep = ''.join(line)
    blocks = re.findall(r'[.O]*(?:#|$)+', rep)
    tilted_blocks = [sorted(b,key=lambda x: '.O#'.index(x)) for b in blocks]
    return np.array(list(itertools.chain(*tilted_blocks)))


class direction():
    north = 3
    east = 0
    south = 1
    west = 2

@np_cache
def tilt(dish, direction):
    lines = np.rot90(dish,k=direction)
    tilted = np.array([tilt_line(l) for l in lines])

    return np.rot90(tilted,k=4-direction)

assert np.array_equal(tilt(parse_input(sample),direction.north),parse_input(sample_tilted_north))
assert np.array_equal(tilt(parse_input(sample),direction.east),parse_input(sample_tilted_east))
assert np.array_equal(tilt(parse_input(sample),direction.west),parse_input(sample_tilted_west))

def cycle(dish):
    res = dish
    for d in [direction.north, direction.west, direction.south, direction.east]:
        res = tilt(res,d)
    return res


def solve1(puzzle):
    dish = parse_input(puzzle)
    tilted = tilt(dish,direction.north)
    return load(tilted)


def solve2(puzzle):
    dish = parse_input(puzzle)
    seen = list()
    for _ in range(1000000000):
        dish=cycle(dish)
        pkl = pickle.dumps(dish)
        if pkl in seen:
            break
        seen.append(pkl)

    len_cycle = len(seen)-seen.index(pkl)
    print(f"found cycle of length {len_cycle}")
    remaining = (1000000000-len(seen)) % len_cycle
    for _ in range(remaining-1):
        dish=cycle(dish)
        #print(load(dish))
    return load(dish)




puzzle = get_input('14')

assert solve1(sample) == 136
assert solve1(puzzle) == 107142
assert solve2(sample) == 64


found cycle of length 7


In [26]:

parsed = parse_input(puzzle)
print(parsed.shape)

(100, 100)


In [27]:
solve1(puzzle)

107142

In [28]:

after_1_cycles = """.....#....
....#...O#
...OO##...
.OO#......
.....OOO#.
.O#...O#.#
....O#....
......OOOO
#...O###..
#..OO#...."""

after_2_cycles=""".....#....
....#...O#
.....##...
..O#......
.....OOO#.
.O#...O#.#
....O#...O
.......OOO
#..OO###..
#.OOO#...O"""

after_3_cycles=""".....#....
....#...O#
.....##...
..O#......
.....OOO#.
.O#...O#.#
....O#...O
.......OOO
#...O###.O
#.OOO#...O"""

assert np.array_equal(cycle(parse_input(sample)),parse_input(after_1_cycles))
assert np.array_equal(cycle(cycle(parse_input(sample))),parse_input(after_2_cycles))
assert np.array_equal(cycle(cycle(cycle(parse_input(sample)))),parse_input(after_3_cycles))

In [29]:
solve2(puzzle)


found cycle of length 22


104815